For the sake of the kernel dying even on the 4gig `ml.t2.medium` sagemaker notebook instance, using my [notebook here](https://github.com/namoopsoo/learn-citibike/blob/2020-revisit/notes/2020-06-28-take2.md) , going to use [the notes on class distribution](https://github.com/namoopsoo/learn-citibike/blob/2020-revisit/notes/2020-06-26.md#looking-at-target-class-distribution) to reduce the size

In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [3]:
tripsdf.shape

(843416, 15)

In [7]:
x = np.random.choice(np.array(range(20)), size=10, replace=False, 
                 p=[1/200 for _ in range(20)])
sorted(x)

ValueError: probabilities do not sum to 1

In [10]:
a = np.array([.1, .2, .3, .4])
average = .25
b = .25/a
new_norm = np.sum(b)
c = b/new_norm
c

array([0.48, 0.24, 0.16, 0.12])

In [18]:
c = fu.rebalance_proportions(a)
near1 = np.sum(c)
#d = c/near1
#sum(d)
print(c, near1)

[0.48 0.24 0.16 0.12] 1.0000000000000002


In [17]:
# ok , looks like np.random.choice is ok with near1 sum..
x = np.random.choice(np.array([1,2,3,4]), size=4, replace=False, 
                 p=c)
sorted(x)

[1, 2, 3, 4]

### 2020-07-01

#### Testing out dataset rebalancing ..
First here creating a random dataset of a predictable proportion , and verifying I can tweak it..

In [26]:

array = np.array([1, 2, 3, 4, 5])
size = array.shape[0]
new_size = 10000
newarray = np.random.choice(array, replace=True,
                size=new_size, p=[.2, .2, .2, .2, .2])
print({k:v/new_size for (k,v) in dict(Counter(newarray)).items()})
print(newarray
     )

{4: 0.1987, 2: 0.1967, 1: 0.2031, 5: 0.2013, 3: 0.2002}
[4 2 1 ... 3 1 4]


In [ ]:
reload(fu); import ipdb
newX, newy = ipdb.runcall(fu.balance_dataset, np.array([1,2]), newarray, shrinkage=.5)

> /opt/program/fresh/utils.py(175)balance_dataset()
    174     # Balance before encoding
--> 175     size = y.shape[0]
    176     dd = list(dict(Counter(y)).items())

ipdb> n
> /opt/program/fresh/utils.py(176)balance_dataset()
    175     size = y.shape[0]
--> 176     dd = list(dict(Counter(y)).items())
    177     counts = dict(Counter(y))

ipdb> 
> /opt/program/fresh/utils.py(177)balance_dataset()
    176     dd = list(dict(Counter(y)).items())
--> 177     counts = dict(Counter(y))
    178     total = sum([x[1] for x in dd])

ipdb> n
> /opt/program/fresh/utils.py(178)balance_dataset()
    177     counts = dict(Counter(y))
--> 178     total = sum([x[1] for x in dd])
    179     dd2 = rebalance_proportions(np.array([x[1]/total for x in dd]))

ipdb> p counts
{4: 1987, 2: 1967, 1: 2031, 5: 2013, 3: 2002}
ipdb> p (np.array([x[1]/total for x in dd]))
*** NameError: name 'total' is not defined
ipdb> n
> /opt/program/fresh/utils.py(179)balance_dataset()
    178     total = sum([x[1] for x 